# Feature Engineering / Ingeniería de Características

En esta libreta, el foco principal, sera realizar la ingeniería de caracteristicas, con el objetivo de crear, inferir y seleccionar caracteristicas a partir de los mismos datos que ayuden al modelo a identificar mejor los patrones y relaciones con la variable objetivo (dependiente) y con esto pueda hacer mejores predicciones y lograr un grado adecuado de generalización.

## 1. Carga de librerias y configuración de libreta

In [1]:
# importar todas las librerias a usar

import pandas as pd
import matplotlib as plt
import mplcyberpunk
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
# Configurar preferencias personales

# Mostrar números sin notación científica
pd.set_option('display.float_format', '{:,.2f}'.format)

# Mostrar todos los separadores de miles para enteros
pd.options.display.float_format = '{:,.2f}'.format  # con 2 decimales

# Usar estilo en gráficos
plt.style.use("cyberpunk")
plt.style.use("dark_background")
plt.rcParams["axes.grid"] = False  # apaga el grid en todas las gráficas

## 2. Carga de archivos

In [3]:
calendar = pd.read_csv('../data/cleaned_calendar.csv', parse_dates=["date"])
inventory = pd.read_csv('../data/inventory.csv')
sales_train = pd.read_csv('../data/sales_train_subset.csv', parse_dates=["date"])

## 3. Validación de carga y tipos de datos

In [4]:
sales_train.head(2)

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount,sales_log,price_log,orders_log,max_discount
0,4845,2024-03-10,Budapest_1,"6,436.00",16.34,646.26,1.00,0.00,0.00,0.00,0.00,0.15,0.00,0.00,2.85,6.47,8.77,0.15
1,4845,2023-04-29,Budapest_1,"5,463.00",34.52,646.26,0.96,0.20,0.00,0.00,0.00,0.15,0.00,0.00,3.57,6.47,8.61,0.20


In [5]:
sales_train.dtypes

unique_id                   int64
date               datetime64[ns]
warehouse                  object
total_orders              float64
sales                     float64
sell_price_main           float64
availability              float64
type_0_discount           float64
type_1_discount           float64
type_2_discount           float64
type_3_discount           float64
type_4_discount           float64
type_5_discount           float64
type_6_discount           float64
sales_log                 float64
price_log                 float64
orders_log                float64
max_discount              float64
dtype: object

In [6]:
inventory.head(2)

,unique_id,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,warehouse
0,5255,2583,Pastry_196,Bakery,Bakery_L2_14,Bakery_L3_26,Bakery_L4_1,Prague_3
1,4948,2426,Herb_19,Fruit and vegetable,Fruit and vegetable_L2_30,Fruit and vegetable_L3_86,Fruit and vegetable_L4_1,Prague_3


In [7]:
inventory.dtypes

unique_id               int64
product_unique_id       int64
name                   object
L1_category_name_en    object
L2_category_name_en    object
L3_category_name_en    object
L4_category_name_en    object
warehouse              object
dtype: object

In [8]:
calendar.head(2)

,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
0,2022-03-16,No_Holiday,0,0,0,0,Frankfurt_1
1,2020-03-22,No_Holiday,0,0,0,0,Frankfurt_1


In [9]:
calendar.dtypes

date                      datetime64[ns]
holiday_name                      object
holiday                            int64
shops_closed                       int64
winter_school_holidays             int64
school_holidays                    int64
warehouse                         object
dtype: object

## 4. Unificación de archivos

In [10]:
del df

NameError: name 'df' is not defined

In [10]:
sales_train.shape

(1765277, 18)

In [11]:
# Unir sales con inventory
df = sales_train.merge(inventory, on="unique_id", how="left")

In [12]:
df.shape

(1765277, 25)

In [13]:
df.head(2).T

,0,1
unique_id,4845,4845
date,2024-03-10 00:00:00,2023-04-29 00:00:00
warehouse_x,Budapest_1,Budapest_1
total_orders,"6,436.00","5,463.00"
sales,16.34,34.52
sell_price_main,646.26,646.26
availability,1.00,0.96
type_0_discount,0.00,0.20
type_1_discount,0.00,0.00
type_2_discount,0.00,0.00


Como la columna de warehouse existe en ambos archivos, se duplica y se asignan sufijo, eliminaré la proveniente del archivo de inventario, pero antes de hacerlo haré una validación rapida de que sean identicas

In [14]:
(df["warehouse_x"] == df["warehouse_y"]).all()

True

Son iguales, por lo que eliminaremos la segunda

In [15]:
df = df.drop(columns=["warehouse_y"]).rename(columns={"warehouse_x": "warehouse"})

Ahora uniremos con el calendario, recordemos que debido a que son distintos en cada país, el calendario varia dependiendo el almacén, por lo que debemos considerar esto en el merge

In [16]:
# Luego unir con calendar (por la fecha)
df = df.merge(calendar, on=["date", "warehouse"], how="left")

In [17]:
df.shape

(1765277, 29)

In [18]:
df.head(2).T

,0,1
unique_id,4845,4845
date,2024-03-10 00:00:00,2023-04-29 00:00:00
warehouse,Budapest_1,Budapest_1
total_orders,"6,436.00","5,463.00"
sales,16.34,34.52
sell_price_main,646.26,646.26
availability,1.00,0.96
type_0_discount,0.00,0.20
type_1_discount,0.00,0.00
type_2_discount,0.00,0.00


## 5. Creación de Features

### Features de calendario

In [19]:
df['year'] = df['date'].dt.year
df['day_of_week'] = df['date'].dt.dayofweek
df['day_of_year'] = df['date'].dt.dayofyear
df["year_month"] = df["date"].dt.to_period("M").astype(str)
df['cos_day'] = np.cos(df['day_of_year']*2*np.pi/365)
df['sin_day'] = np.sin(df['day_of_year']*2*np.pi/365)

- year → captura tendencias a largo plazo, como crecimiento de ventas o inflación  
- day_of_week → incorpora patrones semanales de demanda (ej. más ventas en fin de semana)  
- day_of_year → refleja la estacionalidad anual, útil para productos afectados por temporadas (ej. frutas, pan de temporada)  
- year_month → facilita el análisis y agregación a nivel mensual, capturando variaciones de precios y ventas por periodo  
- cos_day y sin_day → transforman el día del año en variables cíclicas, evitando saltos artificiales entre el 31 de diciembre y el 1 de enero; representan de forma más natural la estacionalidad  

Estos features permiten que el modelo aprenda patrones temporales y estacionales, fundamentales en forecasting de retail.


In [20]:
df.head(2).T

,0,1
unique_id,4845,4845
date,2024-03-10 00:00:00,2023-04-29 00:00:00
warehouse,Budapest_1,Budapest_1
total_orders,"6,436.00","5,463.00"
sales,16.34,34.52
sell_price_main,646.26,646.26
availability,1.00,0.96
type_0_discount,0.00,0.20
type_1_discount,0.00,0.00
type_2_discount,0.00,0.00


### Features de Precio

En este punto, recordemos que hicmos una transformación logarítmica en las variables de precio, ventas y ordenes

In [21]:
df[["unique_id", "date", "sales_log" , "price_log", "orders_log"]].head()


,unique_id,date,sales_log,price_log,orders_log
0,4845,2024-03-10,2.85,6.47,8.77
1,4845,2023-04-29,3.57,6.47,8.61
2,4845,2024-03-02,3.34,6.47,8.82
3,4845,2023-07-04,3.39,6.47,8.67
4,4845,2024-05-11,3.07,6.47,8.77


### Features de promedios móviles

Para el caso de forecast en retail son muy útiles los promedios móviles, y para poderlos calcular, se ordenan los valores agrupando por producto, por lo que antes de correr los calculos, me aseguraré de que los valroes en unique_id no se repitan entre almacenes y evitar asi que se mezcle información

In [22]:
df[df['L1_category_name_en'] == 'Bakery'][["name","warehouse", "unique_id", "product_unique_id"]].drop_duplicates().sort_values(by="name")


,name,warehouse,unique_id,product_unique_id
746785,Bagel_0,Budapest_1,37,20
365494,Bagel_1,Budapest_1,715,368
1275617,Bagel_10,Brno_1,4341,2142
1525094,Bagel_10,Prague_1,4343,2142
1430385,Bagel_10,Prague_3,4342,2142
...,...,...,...,...
934026,Wrap_1,Munich_1,1551,790
1274081,Wrap_2,Budapest_1,2912,1444
1188767,Wrap_3,Frankfurt_1,5009,2457
1250990,Wrap_3,Munich_1,5010,2457


Con la tabla resultante confirmamos esto, por lo que podemos proceder con la creación de los promedios móviles

In [23]:
df = df.sort_values(["unique_id", "date"])

df["sales_rolling_7d"] = (
    df.groupby("unique_id")["sales"]
      .transform(lambda x: x.shift(1).rolling(7, min_periods=1).mean())
)

df["sales_rolling_28d"] = (
    df.groupby("unique_id")["sales"]
      .transform(lambda x: x.shift(1).rolling(28, min_periods=1).mean())
)


In [24]:
df.head(2).T

,1703465,1703473
unique_id,0,0
date,2023-07-12 00:00:00,2023-07-13 00:00:00
warehouse,Budapest_1,Budapest_1
total_orders,"5,741.00","5,739.00"
sales,62.34,65.28
sell_price_main,853.20,853.20
availability,0.95,1.00
type_0_discount,0.00,0.00
type_1_discount,0.00,0.00
type_2_discount,0.00,0.00


In [25]:
# Guardar DataFrame enriquecido a CSV
output_path = "../data/sales_train_enriched.csv"

df.to_csv(output_path, index=False)

print(f"DataFrame guardado en: {output_path}")
print(f"Filas: {len(df)}, Columnas: {len(df.columns)}")


DataFrame guardado en: ../data/sales_train_enriched.csv
Filas: 1765277, Columnas: 37
